首先，我们构造一个没有任何参数的自定义层。如果你还记得我们在 5.1节 对块的介绍，这应该看起来很眼熟。下面的CenteredLayer类要从其输入中减去均值。要构建它，我们只需继承基础层类并实现正向传播功能。

In [1]:
import torch 
import torch.nn.functional as F
from torch import nn

In [4]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,X):
        return X-X.mean()

提供一些数据看其是否正常工作

In [5]:
layer =CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5,6]))

tensor([-2.5000, -1.5000, -0.5000,  0.5000,  1.5000,  2.5000])

现在我们可以把层作为组件合并到构建更复杂的模型中

In [6]:
net =nn.Sequential(nn.Linear(8,128),CenteredLayer())

作为额外的健全性检查，我们可以向网络发送随机数据后，检查均值是否为0。由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非零数。

In [7]:
Y =net(torch.rand(size=(4,8)))
Y.mean()

tensor(2.3283e-09, grad_fn=<MeanBackward0>)

# 带参数的层

## 既然我们知道了如何定义简单的层，那么让我们继续定义具有参数的层，这些参数可以通过训练进行调整。我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。比如管理访问、初始化、共享、保存和加载模型参数。这样做的好处之一是，我们不需要为每个自定义层编写自定义序列化程序。
## 现在，让我们实现自定义版本的全连接层。回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。在此实现中，我们使用ReLU作为激活函数。该层需要输入参数：in_units和units，分别表示输入和输出的数量。

In [8]:
class MyLinear(nn.Module):
    def __init__(self,in_units,out_units):
        super().__init__()
        self.weight =nn.Parameter(torch.randn(in_units,out_units))
        self.bias =nn.Parameter(torch.randn(out_units))
    
    def forward(self,X):
        output =torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(output)

## 接下来实例化这个线性层

In [9]:
linear =MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[-0.4163,  0.2215, -2.6366],
        [-0.6289, -1.5524, -0.2151],
        [ 0.2073,  0.3721, -1.0200],
        [-0.7607,  1.5225,  0.6262],
        [ 1.9960, -0.4492,  1.8965]], requires_grad=True)

## 执行以下正向传播

In [11]:
linear(torch.rand(10,5))

tensor([[0.0000, 0.3280, 0.0478],
        [0.0000, 0.9474, 1.4289],
        [0.0000, 0.8406, 0.1742],
        [0.0000, 0.5416, 1.3683],
        [0.0000, 0.0000, 0.0000],
        [0.0000, 0.5150, 1.8463],
        [0.0000, 0.8029, 0.0000],
        [0.0000, 1.4035, 0.4187],
        [0.0000, 0.0000, 0.1396],
        [0.0000, 1.2605, 0.2553]])

## 还可以使用自定义层构建模型。可以像使用内置层一样使用自定义层

In [16]:
net =nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(size=(5,64)))

tensor([[19.0979],
        [ 8.0067],
        [11.6849],
        [ 6.4183],
        [ 9.0448]])